# Carga de las librerías necesarias
Se importan las librerías necesarias para la ejecución del algoritmo.

Las librerías deben estar previamente instaladas en Python.

In [1]:
%reset -f

import os
import numpy as np
import pandas as pd
import random
import math
from math import e

import configparser

# Lectura y preparación de datos
Función para leer los archivos de datos necesarios para ejecutar el algoritmo.

Los archivos de datos están descritos en la sección "Parametrización" del documento "Formalización del modelo".

Esta función también prepara los datos para su ejecución, es decir, une bases de datos y construye campos adicionales.


In [2]:
def load_data():
    """Lee los datos del directorio de trabajo.
    """
    global pathoutput
    global nodos, inv, vu, cartectraf, cyd
    global HSCD, PVDT, TNTR, TAMB, PDRE, FCMX, FCPR, HVUT, AVUT
    global P1FVAC1, P1FVAC2, P1FCAR1, P1FCAR2, P1FCAR3, P1FCAR4
    global P3FVAC11, P3FVAC12, P3FVAC21, P3FVAC22, P3FVAC31, P3FVAC32
    global P3FCAR11, P3FCAR12, P3FCAR13, P3FCAR21, P3FCAR22, P3FCAR31, P3FCAR32

    #######################################################
    # definicion del archivo txt donde estan los parametros
    parser = configparser.ConfigParser()
    parser.read(working_dir + 'input/params.txt')

    # parametros globales
    HSCD = float(parser['CALCULOS']['horas_sobrecarga_dia'])
    PVDT = float(parser['CALCULOS']['perdida_vida_diaria_teorica'])
    TNTR = float(parser['CALCULOS']['temperatura_normal_trafo'])
    TAMB = float(parser['CALCULOS']['temperatura_ambiente'])
    PDRE = float(parser['CALCULOS']['probabilidad_deterioro_reubicacion'])
    FCMX = float(parser['CALCULOS']['factor_carga_maxima'])
    FCPR = float(parser['CALCULOS']['factor_carga_promedio'])

    # calculos con parametros globales
    HVUT = 24 / PVDT
    AVUT = HVUT / (24*365) 

    # resoluciones 818 y 819
    ## transformadores monofasicos - perdidas en vacio
    P1FVAC1 = float(parser['RES818819']['P1FVAC1'])
    P1FVAC2 = float(parser['RES818819']['P1FVAC2'])

    ## transformadores monofasicos - perdidas con carga
    P1FCAR1 = float(parser['RES818819']['P1FCAR1'])
    P1FCAR2 = float(parser['RES818819']['P1FCAR2'])
    P1FCAR3 = float(parser['RES818819']['P1FCAR3'])
    P1FCAR4 = float(parser['RES818819']['P1FCAR4'])

    ## transformadores trifasicos -- perdidas en vacio
    P3FVAC11 = float(parser['RES818819']['P3FVAC11'])
    P3FVAC12 = float(parser['RES818819']['P3FVAC12'])
    P3FVAC21 = float(parser['RES818819']['P3FVAC21'])
    P3FVAC22 = float(parser['RES818819']['P3FVAC22'])
    P3FVAC31 = float(parser['RES818819']['P3FVAC31'])
    P3FVAC32 = float(parser['RES818819']['P3FVAC32'])

    ## transformadores trifasicos - perdidas con carga
    P3FCAR11 = float(parser['RES818819']['P3FCAR11'])
    P3FCAR12 = float(parser['RES818819']['P3FCAR12'])
    P3FCAR13 = float(parser['RES818819']['P3FCAR13'])
    P3FCAR21 = float(parser['RES818819']['P3FCAR21'])
    P3FCAR22 = float(parser['RES818819']['P3FCAR22'])
    P3FCAR31 = float(parser['RES818819']['P3FCAR31'])
    P3FCAR32 = float(parser['RES818819']['P3FCAR32'])

    #######################################################
    
    # lectura de las tablas de datos
    nodos = pd.read_csv(working_dir + "input/nodos.csv", sep=',', decimal='.')
    inv = pd.read_csv(working_dir + "input/inventario_transformadores.csv", sep=',', decimal='.',encoding = "ISO-8859-1")
    cartectraf = pd.read_csv(working_dir + "input/carac_tecn_transf.csv", sep=',', decimal='.')
    vu = pd.read_csv(working_dir + "input/vida_util.csv", sep=',', decimal='.')
    cyd = pd.read_csv(working_dir + "input/cyd_pornodo.csv", sep=',', decimal='.')

    # limpieza de datos
    # eliminar columnas en blanco
    nodos=nodos.drop(nodos.columns[nodos.columns.str.contains('unnamed',case = False)],axis = 1)
    inv=inv.drop(inv.columns[inv.columns.str.contains('unnamed',case = False)],axis = 1)

    # agregar columna con id_n y id_t
    nodos.insert(0, 'id_n', range(1, 1+len(nodos)))
    inv.insert(0, 'id_t', range(1, 1+len(inv)))

    # eliminar filas que sobran
    nodos=nodos.loc[(nodos.id_nodo.notnull())]
    
    # eliminar las comas
    nodos.columns = [col.replace(',','') for col in nodos.columns]

    ############################################################
    
    # nombres de las columnas de las tablas de datos
    nodos.columns = ['id_n', 'id_n_Internexa','lat','lon','tension','cpro_n','cmax_n','cremcreg','dmda_n','cens','cred','tusu','pkwh_n']
    inv.columns = ['id_t', 'id_t_Internexa','fab','fase_t','tais','capa_t','vprim','vsecu','ffab','anus','viut_t','id_n_Internexa','tacr_t','creu_t','finst']
    vu.columns = ['tgrc', 'fase_t','lipo','lspo','cpre','dura','cpor','tmpc','tmac']
    cartectraf.columns = ['fase_t', 'capa_t','cnue_t']
    cyd.columns = ['id','id_t_Internexa','cmax','cprom','ddadia']

    # señalar la bodega
    nodos.loc[nodos.id_n_Internexa == 'Palmira','id_n']=999999
    nodos.loc[nodos.id_n == 999999,'dmda_n']=0
    # eliminar las comas de los campos con valores
    cols = ['cpro_n','cmax_n','dmda_n','pkwh_n']
    nodos[cols] = nodos[cols].replace({'\$': '', ',': ''}, regex=True)
    nodos[cols] = nodos[cols].apply(pd.to_numeric, errors='coerce', axis=1)
    cols = ['capa_t','ffab','anus','viut_t','creu_t','viut_t','anus']
    inv[cols] = inv[cols].replace({'\$': '', ',': ''}, regex=True)
    inv[cols] = inv[cols].apply(pd.to_numeric, errors='coerce', axis=1)
    
    # agregar la carga maxima y carga promedio de los nodos
    inv.id_t_Internexa = inv.id_t_Internexa.apply(str)
    cyd.id_t_Internexa = cyd.id_t_Internexa.apply(str)
    inv = inv.merge(nodos[['id_n','id_n_Internexa']], on = 'id_n_Internexa',how = 'left')
    inv = inv.merge(cyd[['cmax','cprom','id_t_Internexa']], on = 'id_t_Internexa',how = 'left')
    nodos = nodos.merge(inv.loc[inv.id_n_Internexa != 'Palmira'][['cmax','cprom','id_n_Internexa']], on = 'id_n_Internexa',how = 'left')
    nodos['cpro_n'] = nodos['cprom']
    nodos['cmax_n'] = nodos['cmax']
    nodos.drop(['cprom', 'cmax'], inplace=True, axis=1)
    inv.drop(['cprom', 'cmax'], inplace=True, axis=1)
    # en caso de que no se disponga de informacion historica del comportamiento del nodo se usan indicadores nacionales
    nodos.loc[(nodos['cpro_n'].isnull()) & (nodos['id_n'] != 999999),'cpro_n'] = nodos['dmda_n'] / 30 * FCPR
    nodos.loc[(nodos['cmax_n'].isnull()) & (nodos['id_n'] != 999999),'cmax_n'] = nodos['dmda_n'] / 30 * FCMX

    # indicar grupo del trafo para calculo de las perdidas de transformacion
    inv['grpt_t'] = 1
    inv.loc[(inv.fase_t == 3) & (inv.capa_t >= 150), 'grpt_t'] = 2
    inv.loc[(inv.fase_t == 3) & (inv.capa_t >= 800), 'grpt_t'] = 3

    # indicar grupo del trafo para calculo de las perdidas de vida util
    inv['grpv_t'] = 1
    inv.loc[(inv.fase_t == 1) & (inv.capa_t > 50), 'grpv_t'] = 2
    inv.loc[(inv.fase_t == 3) & (inv.capa_t >= 150), 'grpv_t'] = 2
    inv.loc[(inv.fase_t == 3) & (inv.capa_t >= 500), 'grpv_t'] = 3

    # calcular los años de uso a partir de la fecha de instalación
    ano = inv['finst'].astype(str).str[0:4]
    ano = ano.apply(pd.to_numeric, errors='coerce')
    inv['anus']=2018-a
    #los trafos en operación sin años de uso se asumen con los años de uso promedio de todo el parque de trafos
    inv.loc[inv.anus.isnull(),'anus']=inv.anus.mean()
    #supuesto: todos los trafos de bodega están nuevos
    inv.loc[(inv['finst'].isnull()) & (inv['id_n']==999999),'anus']=0

    # calcular vida util restante del trafo en meses
    inv['viut_t'] = AVUT
    inv['viur_t'] = (inv.viut_t - inv.anus) * 12
    inv.loc[inv.viur_t < 0, 'viur_t'] = 1

    # indicar grupo de vida util
    vu['grpv_t'] = 1
    vu.loc[(vu.fase_t == 1) & (vu.lipo >= 50), 'grpv_t'] = 2
    vu.loc[(vu.fase_t == 3) & (vu.lipo >= 150),'grpv_t'] = 2
    vu.loc[(vu.fase_t == 3) & (vu.lipo >= 500), 'grpv_t'] = 3

    # armar keys para busquedas
    cartectraf['faca'] = cartectraf.fase_t.map(str) + "-" + cartectraf.capa_t.map(str)
    vu['tfcg'] = vu.tgrc.map(str) + "-" + vu.fase_t.map(str) + "-" + vu.cpre.map(str) + "-" + vu['grpv_t'].map(str)
    
    # calculo del costo de cada KVA instalado en la bodega en la solucion inicial
    inv2=inv.copy()
    inv2['faca'] = inv2.fase_t.map(str) + "-" + inv2.capa_t.map(str)
    inv2 = inv2.merge(cartectraf[['cnue_t','faca']], on = 'faca',how = 'left')
    totalcap = inv2['capa_t'].sum()
    totalvalor = inv2['cnue_t'].sum()
    precioKVAbod = int(totalvalor / totalcap)

# Inicialización de una solución
Función para inicializar una solución y armarla de manera completa.

Conserva los trafos asignados a los nodos en alguna iteración anterior, y realiza concatenaciones para obtener los campos claves de la solución.
Indentifica si la solución es diferente respecto a la solución inicial.

Inputs: 
* sol: solución
    
Outputs: sol con nuevos campos:
* id_n: id del nodo [int]
* cmax_n: carga maxima del nodo [KVA]
* cpro_n: carga promedio del nodo [KVA]
* pkwh_n: precio del kWh del nodo [\$ por kWh]
* id_t: id del trafo [int]
* capa_t: capacidad del trafo [kVA]
* grpt_t: grupo al que pertenece el trafo para calculo de perdidas tecnicas [int]
* fase_t: fases del trafo [int]
* viur_t: vida útil restante teórica del trafo [meses]
* grpv_t: grupo al que pertenece el trafo para calculo de perdidas por vida util [int]
* creu_t: costos de reubicacion del trafo [$]
* faca: concatenacion de fases y capacidad del trafo
* cnue_t: costo a nuevo del trafo [\$]
* movtrafo: verifica si el par nodo * trafo estaba presente en la solución inicial (toma valor cero en este caso, y un valor diferente en caso contrario) 

In [3]:
def Inicializacion(sol):
    sol = sol.merge(nodos[['id_n','cmax_n','pkwh_n','cpro_n']], on='id_n', how='left')
    sol = sol.merge(inv[['id_t','capa_t','grpt_t','fase_t','viur_t','grpv_t','creu_t']], on='id_t', how='left')
    sol['faca'] = sol.fase_t.map(str) + "-" + sol.capa_t.map(str)
    sol = sol.merge(cartectraf[['faca','cnue_t']], on='faca', how='left')
    
    # inicializaciones en cero:
    sol['cpt_nt']=0; sol['cpfe_nt']=0; sol['cpcu_nt']=0; sol['futi_nt']=0; sol['pfeW']=0; sol['pcuW']=0; sol['ptrW']=0;
    sol['pnvac']=0; sol['pncar']=0; sol['cperm']=0; sol['cdete']=0; sol['movtrafo']=0; sol['cambioeniter']=0
    sol['cvu_nt']=0; sol['viur_r']=0
    sol['key'] = 0

    # identificar los movimientos de esta solucion respecto a la inicial
    sol['movtrafo'] = sol['id_t'] - solini['id_t']

    return sol

# Costo pérdidas de transformación de una solución
Función para calcular el costo operativo de una solución (de cada par nodo * trafo de la solución).

Este costo está relacionado con los costos de pérdidas de transformación (ecuación 4), que dependen a su vez
de los costos de pérdidas en hierro (ecuación 6) y en cobre (ecuación 7), descritos en el documento "Formalización del modelo".

Inputs: 
* sol: solución
    
Outputs: sol con nuevo campo:
* cpt_nt: costo perdidas de transformacion del par nodo * trafo [\$/mes]


In [4]:
def CostoPerTranSol(sol):
    # calculo de perdidas en vacío y con carga usando los valores de las normas
    sol.loc[((sol.id_n != 999999) & (sol.fase_t == 1)), 'pnvac'] = P1FVAC1 * sol['capa_t'] ** P1FVAC2    
    sol.loc[((sol.id_n != 999999) & (sol.fase_t == 1)), 'pncar'] = P1FCAR1 * sol['capa_t'] ** 3 + P1FCAR2 * sol['capa_t'] ** 2 + P1FCAR3 * sol['capa_t'] + P1FCAR4
    sol.loc[((sol.id_n != 999999) & (sol.fase_t == 3) & (sol.grpt_t == 1)), 'pnvac'] = P3FVAC11 * sol['capa_t'] ** P3FVAC12
    sol.loc[((sol.id_n != 999999) & (sol.fase_t == 3) & (sol.grpt_t == 1)), 'pncar'] = P3FCAR11 * sol['capa_t'] ** 2 + P3FCAR12 * sol['capa_t'] + P3FCAR13
    sol.loc[((sol.id_n != 999999) & (sol.fase_t == 3) & (sol.grpt_t == 2)), 'pnvac'] = P3FVAC21 * sol['capa_t'] ** P3FVAC22
    sol.loc[((sol.id_n != 999999) & (sol.fase_t == 3) & (sol.grpt_t == 2)), 'pncar'] = P3FCAR21 * sol['capa_t'] + P3FCAR22
    sol.loc[((sol.id_n != 999999) & (sol.fase_t == 3) & (sol.grpt_t == 3)), 'pnvac'] = P3FVAC31 * sol['capa_t'] ** P3FVAC32
    sol.loc[((sol.id_n != 999999) & (sol.fase_t == 3) & (sol.grpt_t == 3)), 'pncar'] = P3FCAR31 * sol['capa_t'] + P3FCAR32
    
    # calculo de perdidas en hierro y cobre, en unidades W
    # ecuaciones 6 y 7 del documento "Formalización del modelo"
    sol.loc[(sol.id_n != 999999),'futi_nt'] = sol['cmax_n']/sol['capa_t']
    sol.loc[(sol.id_n != 999999),'pfeW'] = sol['pnvac']
    sol.loc[(sol.id_n != 999999),'pcuW'] = sol['pncar'] * sol['futi_nt'] ** 2
    sol.loc[(sol.id_n != 999999),'ptrW'] = sol['pfeW'] + sol['pcuW']
    
    # monetizacion de las perdidas, en unidades $$
    # ecuación 4 del documento "Formalización del modelo"
    sol.loc[(sol.id_n != 999999),'cpfe_nt'] = sol['pfeW'] / 1000 * sol['pkwh_n'] * 24 * 30
    sol.loc[(sol.id_n != 999999),'cpcu_nt'] = sol['pcuW'] / 1000 * sol['pkwh_n'] * 24 * 30
    sol['cpt_nt'] = sol['cpfe_nt'] + sol['cpcu_nt']

    return sol

# Costo de pérdida de vida útil de una solución
Función para calcular el costo de la pérdida de vida útil de una solución (de cada par nodo * trafo de la solución).

Este costo está relacionado con la ecuación 9 del documento "Formalización del modelo".

Inputs: 
* sol: solución
    
Outputs:
* sol: solución

In [5]:
def CosVidUt(sol):    
    # calculo de la carga precedente
    # ecuación 12 del documento "Formalización del modelo"
    sol['cpreini_nt']=0
    sol['cpre_nt']=0.9
    sol.loc[(sol.id_n != 999999),'cpre_nt'] = sol['cpro_n']/sol['capa_t']

    # aproximar la carga precedente a los valores de la norma GTC50
    sol.loc[(sol.cpreini_nt < (0.75 + 0.9) / 2),'cpre_nt'] = 0.75
    sol.loc[(sol.cpreini_nt < (0.5 + 0.75) / 2),'cpre_nt'] = 0.5

    # calcular el porcentaje diario de perdida de vida util real en porcentaje
    # ecuaciones 10 y 11 del documento "Formalización del modelo"
    sol['theta'] = teta(sol)
    sol['fevej'] = (HSCD / 24) * (e**(15000/383 - 15000/(sol['theta'] + 273))-1)
    sol['pvdr'] = PVDT * (1 + sol['fevej'])    

    # calcular la vida útil restante real del trafo en ese nodo (en meses)    
    sol.loc[(sol.id_n != 999999),'viur_r'] = 12 / (sol['pvdr'] * 365)
    
    # valorar perdida de vida util restante en pesos, durante lo que queda de vida util del trafo en el nodo 
    # ecuación 9 del documento "Formalización del modelo"
    sol.loc[(sol.id_n != 999999),'cvu_nt'] = sol['cnue_t'] * sol['pvdr'] * 30

    return sol

# Temperatura de los trafos en los nodos de una solución
Función para calcular la temperatura máxima que logran los trafos en los nodos de una solución (de cada par nodo * trafo de la solución).

Esta función es usada por la función CosVidUt en su cálculo de los costos por pérdida de vida útil.

Inputs: 
* sol: solución
    
Outputs:
* sol['theta']: columna 'theta' de la solución, en la que se almacenan las tempraturas

In [6]:
def teta(sol):
    sol['key'] = str(int(TAMB)) + '-' + sol.fase_t.map(str) + '-' + sol.cpre_nt.map(str) + '-' + sol['grpv_t'].map(str)
    
    # asignacion para la primera iteracion
    if parejaseval == 0: sol['theta']=0
    
    # calculo de la temperatura
    for i in range(sol.shape[0]):
        if sol['id_n'].iloc[i] != 999999:
            theta = sol['theta'].iloc[i]
            if (parejaseval == 0 or sol['movtrafo'].iloc[i] != 0):
                key = str(sol['key'].iloc[i])
                tabvu = vu.loc[(vu.tfcg == key) & (vu.dura <= HSCD)]
                ncargas = tabvu.shape[0]
                carga=sol['futi_nt'].iloc[i] * 100
                theta=0
                if carga < tabvu['cpor'].min(): theta = TNTR
                if carga >= tabvu['cpor'].max(): theta = tabvu['tmpc'].max()
                if theta == 0:
                    tabvu = tabvu.sort_values(['cpor'],ascending=[False])
                    for index, row in tabvu.iterrows():
                        if carga <= row['cpor']:
                            theta = row['tmpc']
                            break
            sol.at[i,'theta'] = theta 
    return sol['theta']

# Costos de permutación y deterioro de una solución
Función para calcular los costos de permutación y deterioro en los que se incurre por implementar una solución.
Los costos se calculan para cada par nodo * trafo de la solución.

Corresponde con la ecuación 13 del documento "Formalización del modelo".

Inputs: 
* sol: solución
    
Outputs: sol con dos nuevos campos:
* cdete: costo del deterioro [\$/mes]
* cperm: costo de la permutación [\$/mes]

In [7]:
def CosPermutDete(sol):
    # ecuacion 13 del documento "Formalización del modelo"
    sol.loc[(sol.movtrafo != 0), 'cdete'] = sol['cnue_t'] * PDRE / sol['viur_t']
    sol.loc[(sol.movtrafo != 0), 'cperm'] = sol['creu_t'] / sol['viur_t']
    return sol

# Selección del nodo más costoso
Función para seleccionar el más costoso de los nodos, a quien se le buscará posteriormente un trafo menos costoso para intercambio.

Inputs: 
* sol: solución

Outputs: sol con dos nuevos campos:
* id_n: id del nodo más costoso [int]
* id_t: id del trafo asociado al nodo más costoso en la solución actual [int]

In [8]:
def ndmascostoso(sol):
    # seleccionar los nodos potenciales: no evaluados y no bodega
    solcopia = sol.copy()
    solcopia = solcopia[solcopia['eval'] == 0 & (solcopia['id_n'] != 999999)]    
    solcopia = solcopia.sort_values(['csol'],ascending=[False])

    # identificar el nodo más costoso y su trafo asociado
    id_n = int(solcopia['id_n'].iloc[0])
    id_t = int(solcopia['id_t'].iloc[0])
    
    # señalar en la solución actual que ese nodo ya fue seleccionado para evaluación
    sol.at[sol.id_n == id_n,'eval'] = 1
    
    return (id_n, id_t)

# Identificación de trafos en evaluación en la bodega
Función para identificar los trafos típicos que hay en bodega, a fin de de que la heurística sea más eficiente en la selección de trafos potenciales para un nodo (evitando repetir trafos de iguales características en bodega).

Dos trafos se consideran idénticos (dentro del mismo grupo) cuando tienen igual capacidad, fases y vida útil restante.

Inputs: 
* sol: solución

Outputs:
* sol: solución, en la que se encuentran señalados los trafos sujetos a evaluación (columna 't_enev')


In [9]:
def SeleccTfBodega(sol):
    solprov = sol.copy()
    solprov['gbod']=0

    # guarda los trafos en operación y los señala como "en evaluación", y los trafos en bodega y los señala como "no en evaluación"
    operacion = solprov[solprov.id_n != 999999].copy()
    operacion['t_enev']=1    
    enbodega = solprov[solprov.id_n ==999999].copy()
    enbodega['t_enev']=0

    # conforma grupos por trafos típicos de la bodega (en función de la capacidad, las fases y la vida útil restante)
    grupos = enbodega.groupby(['capa_t','fase_t','viur_t'], as_index=False).mean()

    # asigna un consecutivo a cada grupo conformado
    grupos['gbod'] = range(1, 1+len(grupos))

    # eliminar columna anterior que indica el grupo en bodega
    enbodega = enbodega.drop('gbod', 1)
    
    # indica a cuál grupo pertenece cada trafo que hay en bodega
    enbodega = enbodega.merge(grupos[['capa_t','fase_t','viur_t','gbod']],on=['capa_t','fase_t','viur_t'],how='left')
    
    # ordena por grupo
    enbodega = enbodega.sort_values(['gbod'],ascending=[True])

    # señala con un 1 aquellos trafos de bodega que serán considerados para evaluación (los primeros en su grupo)
    for i in range(len(grupos)):
        enbodega['t_enev'].at[enbodega[enbodega.gbod == i+1].index[0]] = 1

    # arma de nuevo la solución, juntando los trafos en operación y en bodega
    solprov = pd.concat([operacion,enbodega],axis=0,sort=False)
    solprov = solprov.sort_values(['id_n','id_t'],ascending=[True,True])
    sol = solprov.copy()
    
    return sol

# Selección del trafo más costoso
Función para seleccionar el trafo más costoso y hacer intercambio con el trafo del nodo más costoso.

Si el intercambio hace que la solución completa resulte menos costosa, entonces se realiza.

Si el intercambio hace que la solución completa resulte más costosa, se selecciona el segundo trafo más costoso
y se repite el proceso.

El proceso continúa hasta que se encuentra un trafo para hacer intercambio o se evaluan todos los trafos.


Inputs: 
* id_n: id del nodo más costoso [int]
* id_t: id del trafo asociado al nodo más costoso en la solución actual [int]
* sol: solución

Outputs:
* sol: solución, en la que se encuentra el intercambio hecho (si se realiza alguno)


In [11]:
def tfmascostoso(id_n,id_t,sol):
    global avance, control, parejaseval
    
    faset = int(sol.loc[(sol['id_t'] == id_t),'fase_t'])
    capat = float(sol.loc[(sol['id_t'] == id_t),'capa_t'])
    fc = str(faset) + '-' + str(capat)
    sol = SeleccTfBodega(sol)
    pot = sol.loc[(sol['eval'] == 0) & (sol['t_enev'] == 1)]
    pot = pot.loc[(pot['faca'] != fc)]
    tfpot = pot.shape[0]
    ctsol = sol['csol'].sum()
    for i in range(tfpot):
        solcopia=sol.copy()
        parejaseval += 1
        
        # seleccionar el primero de los trafos para hacer intercambio
        id_n2 = pot['id_n'].iloc[i]
        id_t2 = pot['id_t'].iloc[i]
            
        # hacer intercambio temporal
        solcopia.at[(solcopia.id_n == id_n) & (solcopia.id_t == id_t),'id_t'] = id_t2
        solcopia.at[(solcopia.id_n == id_n2) & (solcopia.id_t == id_t2),'id_t'] = id_t
        
        #calcular costos
        solev = solcopia[['id_n','id_t','eval','theta']].copy()
        solev = Inicializacion(solev)
        solev = CosPermutDete(solev)
        solev = CostoPerTranSol(solev)
        solev = CosVidUt(solev)
        solev['csol'] = solev['cpt_nt'] + solev['cvu_nt'] + solev['cperm'] + solev['cdete']
        ctsolcopia = solev['csol'].sum()
        
        # hacer intercambio oficial
        if ctsolcopia < ctsol:
            id_t = id_t2
            sol = solev.copy()
            ctsol = ctsolcopia
            avance.loc[-1] = [giro, parejaseval, ctsol]
            avance.index = avance.index + 1
        control.loc[-1] = [giro, parejaseval, ctsol]
        control.index = control.index + 1
    return sol 

# Bodega
Función para calcular los cambios ocurridos en la bodega una vez se obtiene la solución final.

Inputs: 
    
Outputs: sol con dos nuevos campos:
* capbodini: capacidad de la bodega en la solución inicial [KVA]
* capbodfin: capacidad de la bodega en la solución final [KVA]
* caplib: capacidad liberada en la bodega con la solución final [KVA]
* valorlib: monetización de la capacidad liberada en la bodega con la solución final [$]

In [12]:
def bodega():
    # precio del VA en bodega
    inv2=inv.copy()
    inv2['faca'] = inv2.fase_t.map(str) + "-" + inv2.capa_t.map(str)
    inv2 = inv2.merge(cartectraf[['cnue_t','faca']], on = 'faca',how = 'left')
    totalcap = inv2['capa_t'].sum()
    totalvalor = inv2['cnue_t'].sum()
    precioKVAbod = int(totalvalor / totalcap)

    # calculo de la capacidad liberada y valoracion
    bodini = solinicial[solinicial.id_n == 999999]
    bodfin = solfin[solfin.id_n == 999999]
    capbodini = bodini['capa_t'].sum()
    tfbodini = bodini['capa_t'].count()
    capbodfin = bodfin['capa_t'].sum()
    tfbodfin = bodfin['capa_t'].count()
    caplib = capbodfin - capbodini
    valorlib = int(caplib * precioKVAbod)
    return (capbodini,capbodfin,caplib,valorlib)

# Run
Función que contiene la lógica de la heurística.

La función guarda en csv los resultados conforme los va obteniendo.
Cada vez que encuentra una mejor solución, almacena los resultados en csv.

Inputs: 

Outputs:


In [13]:
def run():
   # armar la solucion inicial
    global sol, soliter, solini, solinicial, solfin, ctsol, avance, control, giro, parejaseval
    
    parejaseval=0
    solini=pd.DataFrame(columns=['id_n','id_t'])
    control = pd.DataFrame(columns=['giro','parejas','csol'])
    avance = pd.DataFrame(columns=['giro','parejas','csol'])
    
    solini['id_n'] = inv['id_n']
    solini['id_t'] = inv['id_t'] 
    
    # calcular los costos de la solucion inicial
    sol = Inicializacion(solini)
    
    sol = CosPermutDete(sol)
    sol = CostoPerTranSol(sol)
    sol = CosVidUt(sol)
    sol['csol'] = sol['cpt_nt'] + sol['cvu_nt'] + sol['cperm'] + sol['cdete']
    sol['eval']=0
    sol.to_csv(pathoutput + 'solini.csv')
    solinicial = sol.copy()
    ctsol = sol['csol'].sum()
    
    avance.loc[-1] = [0, parejaseval, ctsol]
    avance.index = avance.index + 1
    control.loc[-1] = [0,parejaseval, ctsol]
    control.index = control.index + 1

    # buscar el nodo mas costoso
    parejaseval = 0
    giro = 0
    
    soliter = sol.copy()
    stop = 0
    while stop == 0:
        giro += 1
        ndpend = 1
        while (ndpend > 0):
            id_n, id_t = ndmascostoso(sol)
            sol = tfmascostoso(id_n,id_t,sol)
            ndpend = sol.loc[(sol['eval'] == 0) & ((sol['id_n'] != 999999))].shape[0]
            
            #Hacer las gráficas
            #graficar()
            
            #Imprimir porvisionales
            solfin = sol.copy()
            solfin.to_csv(pathoutput + 'solfin.csv')
            avance.to_csv(pathoutput + 'avance.csv')
            control.to_csv(pathoutput + 'control.csv')    
    
    
        #mirar si esta solucion es diferente de la de la iteracion anterior
        sol = sol.sort_values(['id_n', 'id_t'],ascending=[True,True])
        sol = sol.reset_index(drop=True)
        soliter = soliter.reset_index(drop=True)
        sol['cambioeniter'] = (sol['id_t'] - soliter['id_t'])**2
        if sol['cambioeniter'].sum() == 0: stop = 1
        soliter = sol.copy()
        
        #imprimir giro
        print(giro,stop)
        
        #reiniciar la evaluacion
        sol['eval'] = 0        


In [14]:
working_dir="../Tests/Test1/"
pathoutput = working_dir + 'output/'

load_data()
nodos.at[:,'cmax_n'] = nodos.loc[:,'cmax_n']
nodos.at[:,'cpro_n'] = nodos.loc[:,'cpro_n']
run()

AttributeError: 'DataFrame' object has no attribute 'id_nodo'

In [202]:
# impresiones para interrupcion abrupta del codigo
solfin = sol.copy()
solfin.to_csv(pathoutput + 'solfin.csv')
avance.to_csv(pathoutput + 'avance.csv')
control.to_csv(pathoutput + 'control.csv')    

            

In [208]:
inv.to_csv(pathoutput + 'inv.csv')

In [ ]:
#######################

In [210]:
#### calculos de ahorros y mejoras en la solucion
capbodini,capbodfin,caplib,valorlib = bodega()
cini = int(solinicial['csol'].sum())
cfin = int(solfin['csol'].sum())
ahorroop = cini-cfin

print('capacidad total instalada en bodega - solución inicial: ' + str(capbodini) + ' KVA')
print('capacidad total instalada en bodega - solución final: ' + str(capbodfin) + ' KVA')
print('capacidad liberada en bodega: ' + str(caplib) + ' KVA')
print('valor de la capacidad liberada en bodega: ' + str(valorlib) + ' $Col')

print('costo de la solucion inicial = ' + str(cini) + ' $/mes')
print('costo de la solucion final = ' + str(cfin) + ' $/mes')
print('ahorro operativo de la solucion = ' + str(ahorroop) + ' $/mes')
print('ahorro total de la solucion = ' + str(ahorroop) + ' $/mes')


capacidad total instalada en bodega - solución inicial: 5087.5 KVA
capacidad total instalada en bodega - solución final: 6267.5 KVA
capacidad liberada en bodega: 1180.0 KVA
valor de la capacidad liberada en bodega: 77360800 $Col
costo de la solucion inicial = 11150536 $/mes
costo de la solucion final = 10194776 $/mes
ahorro operativo de la solucion = 955760 $/mes
ahorro total de la solucion = 955760 $/mes
